In [8]:
"""
test.ipynb used for testing models with test*.csv

This notebook performs a trailing window on specified time series data with the following parameters:
- window size
- hops
In our context, our data is collected on an average of 1 second per movement. Thus, setting our window
 frame as 1s, with a sampling rate of 50Hz would result in 50 samples per window.
"""

import pandas as pd
import tensorflow as tf
import numpy as np
import xarray

##change sampling parameters here
total_samples = 3100*22*4
window = 100
overlap = 50
num_gestures = 4
data_columns = 6

In [2]:
"""
Sliding window function and params
"""
from tqdm import tqdm
from skimage.util.shape import view_as_windows
import warnings

csv = 'Mervin Test.csv'

def sliding_window(a, L, overlap=1):
    if L==overlap:
        raise Exception("Overlap arg must be smaller than length of windows")
    S = L - overlap
    nd0 = ((len(a)-L)//S)+1
    if nd0*S-S!=len(a)-L:
        warnings.warn("Not all elements were covered")
    output = view_as_windows(a, (L,a.shape[1]), step=S)[:,0,:,:]
    print(output.shape)
    return output

total_array = np.empty((0,window,6))

print('start!')
df = pd.read_csv(csv)
window_array = sliding_window(df.to_numpy(), window, overlap=overlap)
total_array = np.append(total_array, window_array, axis = 0)
print('ok!')
total_array

start!
(60, 100, 6)
ok!


array([[[-1.6000e-01,  2.4000e-01, -1.1700e+00, -1.7240e+01,
         -5.9320e+01,  5.3300e+00],
        [-1.3000e-01,  3.1000e-01, -1.0800e+00,  1.5180e+01,
         -4.5380e+01, -7.0000e-01],
        [-3.0000e-02,  4.7000e-01, -1.0300e+00,  1.2030e+01,
         -6.4000e+01,  1.2300e+00],
        ...,
        [-8.7000e-01,  2.0500e+00, -4.0000e+00, -1.8600e+00,
         -5.2300e+00,  1.6240e+01],
        [-1.2700e+00,  2.0200e+00, -4.0000e+00,  1.0570e+01,
          7.8480e+01,  2.0840e+01],
        [-1.3000e+00,  2.0300e+00, -4.0000e+00,  1.9780e+01,
          1.4068e+02,  6.5530e+01]],

       [[-1.4400e+00,  2.0700e+00, -3.7700e+00,  5.9460e+01,
         -2.5000e+02, -7.6970e+01],
        [-1.2000e+00,  1.8900e+00, -4.0000e+00,  2.6570e+01,
         -2.0087e+02, -3.1170e+01],
        [-9.3000e-01,  1.6600e+00, -4.0000e+00, -4.5410e+01,
         -1.4078e+02, -8.3300e+00],
        ...,
        [-2.0800e+00,  6.4000e-01, -1.4800e+00, -5.3950e+01,
          2.4999e+02,  1.1289e+02],
  

In [3]:
total_array.shape

(60, 100, 6)

In [4]:
"""
Save into txt file
"""
arr = total_array
  
# reshaping the array from 3D 
# matrice to 2D matrice. 
arr_reshaped = arr.reshape(arr.shape[0], -1) 
  
# saving reshaped array to file. 
np.savetxt("Mervin_test_data.txt", arr_reshaped) 
  
# retrieving data from file. 
loaded_arr = np.loadtxt("Mervin_test_data.txt") 
  
# This loadedArr is a 2D array, therefore 
# we need to convert it to the original 
# array shape.reshaping to get original 
# matrice with original shape. 
load_original_arr = loaded_arr.reshape( 
    loaded_arr.shape[0], loaded_arr.shape[1] // arr.shape[2], arr.shape[2]) 
  
# check the shapes: 
print("shape of arr: ", arr.shape) 
print("shape of load_original_arr: ", load_original_arr.shape) 
  
# check if both arrays are same or not: 
if (load_original_arr == arr).all(): 
    print("Yes, both the arrays are same") 
else: 
    print("No, both the arrays are not same") 

shape of arr:  (60, 100, 6)
shape of load_original_arr:  (60, 100, 6)
Yes, both the arrays are same


# Testing the model

In [5]:
from keras.models import load_model

model_name = '../lstm_models/lstm_model_100_50_96%'

model = load_model(model_name)

In [9]:
loaded_arr2 = np.loadtxt('Mervin_test_data.txt')
x_test = loaded_arr2.reshape(loaded_arr2.shape[0], loaded_arr2.shape[1] // data_columns, data_columns)

In [10]:
y_pred = model.predict(x_test)

In [12]:
rounded = np.round(y_pred)

print(y_pred[0])
print(rounded[0])

[9.99856234e-01 6.27867337e-07 2.84920661e-05 1.14642266e-04]
[1. 0. 0. 0.]


In [13]:
if num_gestures == 4:
    class_names = ["Buddha clap", "Knob", "Pushback", "Swipe"]
elif num_gestures == 5:
    class_names = ["Buddha clap", "Crank", "Knob", "Pushback", "Swipe"]
elif num_gestures == 7:
    class_names = ["Buddha clap", "Crank left", "Crank right", "Knob left", "Knob right", "Pushback", "Swipe"]

name = np.array(class_names)
print(name)
print(type(name))

['Buddha clap' 'Knob' 'Pushback' 'Swipe']
<class 'numpy.ndarray'>


In [14]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(name.reshape(-1, 1))
y_pred = enc.inverse_transform(rounded)

In [15]:
print(y_pred)

[['Buddha clap']
 ['Buddha clap']
 ['Buddha clap']
 ['Buddha clap']
 ['Buddha clap']
 ['Buddha clap']
 ['Buddha clap']
 ['Buddha clap']
 ['Swipe']
 ['Swipe']
 ['Swipe']
 ['Swipe']
 ['Swipe']
 ['Swipe']
 ['Swipe']
 ['Swipe']
 ['Knob']
 ['Knob']
 ['Knob']
 ['Knob']
 ['Knob']
 ['Knob']
 ['Knob']
 ['Knob']
 ['Knob']
 ['Swipe']
 ['Swipe']
 ['Swipe']
 ['Buddha clap']
 ['Swipe']
 ['Swipe']
 ['Swipe']
 ['Swipe']
 [None]
 ['Pushback']
 ['Pushback']
 ['Pushback']
 ['Pushback']
 ['Pushback']
 ['Pushback']
 ['Pushback']
 ['Pushback']
 ['Pushback']
 ['Pushback']
 ['Pushback']
 [None]
 ['Swipe']
 ['Swipe']
 ['Swipe']
 ['Swipe']
 ['Swipe']
 ['Swipe']
 ['Swipe']
 ['Swipe']
 ['Swipe']
 ['Buddha clap']
 ['Buddha clap']
 ['Pushback']
 ['Buddha clap']
 ['Swipe']]


In [16]:
output = []

for i in range(len(y_pred)):
        output.append(y_pred[i])
        print("Prediction: %s" % (y_pred[i]))

Prediction: ['Buddha clap']
Prediction: ['Buddha clap']
Prediction: ['Buddha clap']
Prediction: ['Buddha clap']
Prediction: ['Buddha clap']
Prediction: ['Buddha clap']
Prediction: ['Buddha clap']
Prediction: ['Buddha clap']
Prediction: ['Swipe']
Prediction: ['Swipe']
Prediction: ['Swipe']
Prediction: ['Swipe']
Prediction: ['Swipe']
Prediction: ['Swipe']
Prediction: ['Swipe']
Prediction: ['Swipe']
Prediction: ['Knob']
Prediction: ['Knob']
Prediction: ['Knob']
Prediction: ['Knob']
Prediction: ['Knob']
Prediction: ['Knob']
Prediction: ['Knob']
Prediction: ['Knob']
Prediction: ['Knob']
Prediction: ['Swipe']
Prediction: ['Swipe']
Prediction: ['Swipe']
Prediction: ['Buddha clap']
Prediction: ['Swipe']
Prediction: ['Swipe']
Prediction: ['Swipe']
Prediction: ['Swipe']
Prediction: [None]
Prediction: ['Pushback']
Prediction: ['Pushback']
Prediction: ['Pushback']
Prediction: ['Pushback']
Prediction: ['Pushback']
Prediction: ['Pushback']
Prediction: ['Pushback']
Prediction: ['Pushback']
Prediction

In [18]:
import csv
from datetime import datetime

fields = ['Predicted']
now = datetime.now()
file = 'output ' + now.strftime("%d-%m-%Y %H-%M-%S") + '.csv'
with open(file, 'w', newline='') as filename:
    write = csv.writer(filename)
    write.writerow(fields)
    write.writerows(output)
